In [1]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.4 MB/s 
     |████████████████████████████████| 98 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 42.0 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 8.8 MB 10.7 MB/s 
     |████████████████████████████████| 3.0 MB 48.6 MB/s 
     |████████████████████████████████| 3.3 MB 31.1 MB/s 
     |████████████████████████████████| 99 kB 8.6 MB/s 
     |████████████████████████████████| 95 kB 1.0 MB/s 
     |████████████████████████████████| 187 kB 44.6 MB/s 
     |████████████████████████████████| 1.3 MB 39.3 MB/s 
     |████████████████████████████████| 130 kB 49.1 MB/s 
     |████████████████████████████████| 128 kB 54.2 MB/s 
     |████████████████████████████████| 428 kB 48.9 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 34

In [2]:
import ee
import geemap

# ee.Authenticate()
# ee.Initialize()

### Create an interactive map

In [3]:
Map = geemap.Map()
Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Z2flZGdjJzTi9Frn2_Nat0F2sLNwZrCUnhxzFuijkto&tc=0jYP2BZKuT7uIyT5Hovp6ucz0TF48X03AUzHy2OEBRw&cc=4wnPbSHqOscQNtlj_Pl9vWamR3SYeMU161yo9t05N5E

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgDC6LcZaEnsnsQLnpji6LgGJqeLFoorW7vY47kK6hmIQf_MO7jIjM

Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Add data to the map

In [4]:
point = ee.Geometry.Point([-80.55, 43])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [5]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-06-16'

In [6]:
image.get('CLOUD_COVER').getInfo()

0.47

### import AAFC label dataset



In [7]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

use the [Canada AAFC Annual Crop Inventory](https://developers.google.com/earth-engine/datasets/catalog/AAFC_ACI#bands) to create label dataset for training


![](https://www.researchgate.net/profile/Meisam-Amani/publication/345082789/figure/fig1/AS:952723008651265@1604158274050/Annual-Space-based-Crop-Inventory-ACI-produced-by-Agriculture-and-Agri-Food-Canada.png)


In [8]:
# nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry())

affc = ee.ImageCollection('AAFC/ACI').filter(ee.Filter.date('2020-01-01', '2020-12-31')).first().clip(image.geometry())

Map.addLayer(affc, {}, 'AAFC')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

## Prepare for consecutive class labels

In [9]:
raw_class_values = affc.get('landcover_class_values').getInfo()

class_palette = affc.get('landcover_class_palette').getInfo()



raw_class_values

[10,
 20,
 30,
 34,
 35,
 50,
 80,
 85,
 110,
 120,
 122,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 167,
 168,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 210,
 220,
 230]

In [10]:
initial_train_accuracy = [[0],
 [0.8996450467892869],
 [0.00819672131147541],
 [0.17928902627511592],
 [0],
 [0.1495766698024459],
 [0.1269968051118211],
 [0],
 [0],
 [0],
 [0.6437947494033412],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.055793991416309016],
 [0],
 [0.13953488372093023],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.13537117903930132],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0.2768595041322314],
 [0.6237767306995288],
 [0.7143840330350998]]

In [11]:
reduced_class_values = raw_class_values
reduced_class_palette = class_palette

for i in range(len(class_palette)):
  if initial_train_accuracy[i][0] == 0:
    reduced_class_values[i] = 0
    reduced_class_palette[i] = 0

reduced_class_values = [i for i in reduced_class_values if i != 0]
reduced_class_palette = [i for i in reduced_class_palette if i != 0]

In [12]:
reduced_class_values

[20, 30, 34, 50, 80, 122, 145, 147, 158, 210, 220, 230]

In [13]:
n_classes = len(reduced_class_values)
new_class_values = list(range(0, n_classes))

In [14]:
affc = affc.remap(reduced_class_values, new_class_values).select(
    ['remapped'], ['landcover']
)
affc = affc.set('landcover_class_values', new_class_values)
affc = affc.set('landcover_class_palette', reduced_class_palette)

## Make  dataset

In [15]:
# Make the training dataset.
points = affc.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 15000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)


## Split training and validation

In [16]:
points = points.randomColumn();

training_points = points.filter('random <= 0.7');
validation_points = points.filter('random > 0.7');

Map.addLayer(training_points, {}, 'training', False)
Map.addLayer(validation_points, {}, 'validation', False)

In [17]:
print(training_points.size().getInfo())

8994


In [18]:
print(training_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-79.93783236408851, 42.70212558121815]}, 'id': '0', 'properties': {'landcover': 0, 'random': 0.36042193558372604}}


### Train the classifier

In [19]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': training_points, 'properties': [label], 'scale': 30}
)

validation = image.select(bands).sampleRegions(
    **{'collection': validation_points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trainedClassifier = ee.Classifier.smileRandomForest(27).train(training, label, bands)

In [20]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 319, 'B2': 324, 'B3': 273, 'B4': 149, 'B5': 116, 'B6': 93, 'B7': 76, 'landcover': 0}}


# Accuracy assessment
## Training dataset

In [21]:
# Get a confusion matrix and overall accuracy for the training sample.
train_accuracy = trainedClassifier.confusionMatrix();
# trainAccuracy.getInfo()

###Training - Overall Accuracy

In [22]:
train_accuracy.accuracy().getInfo()

0.9840898976412995

###Training -  Kappa

In [23]:
train_accuracy.kappa().getInfo()

0.9810472905405722

###Training - Producer's Accuracy

In [24]:
train_accuracy.producersAccuracy().getInfo()

[[1],
 [0.8876404494382022],
 [0.9914367269267365],
 [0.9819819819819819],
 [0.9404761904761905],
 [0.9828326180257511],
 [0.9655797101449275],
 [0.9836734693877551],
 [0.9729448491155047],
 [0.95],
 [0.9900426742532006],
 [0.927461139896373]]

###Training - Comsumer's Accuracy

In [25]:
train_accuracy.consumersAccuracy().getInfo()


[[0.9966355140186915,
  1,
  0.9774859287054409,
  0.990909090909091,
  0.9916317991631799,
  0.9662447257383966,
  0.9944029850746269,
  0.9796747967479674,
  0.9831756046267087,
  1,
  0.9680111265646731,
  1]]

# Validation dataset

In [26]:
validated = validation.classify(trainedClassifier)

In [27]:
test_accuracy = validated.errorMatrix('landcover', 'classification')

In [28]:
# test_accuracy.getInfo()

## Validation - Overall Accuracy


In [29]:
test_accuracy.accuracy().getInfo()

0.727432296890672

## Validation - Kappa

In [30]:
test_accuracy.kappa().getInfo()

0.6777328218141782

## Validation - Producer's Accuracy

In [31]:
test_accuracy.producersAccuracy().getInfo()

[[0.9990485252140818],
 [0.16666666666666666],
 [0.7398190045248869],
 [0.23636363636363636],
 [0.25],
 [0.75],
 [0.7008196721311475],
 [0.6728813559322034],
 [0.4900662251655629],
 [0],
 [0.7049689440993789],
 [0.23333333333333334]]

## Validation - Consumers’ Accuracy

In [32]:
test_accuracy.consumersAccuracy().getInfo()

[[0.9924385633270322,
  0.35,
  0.6673469387755102,
  0.3611111111111111,
  0.38461538461538464,
  0.6185133239831697,
  0.7566371681415929,
  0.6455284552845528,
  0.6149584487534626,
  0,
  0.6776119402985075,
  0.30434782608695654]]

### Classify the image

In [33]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trainedClassifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [34]:
class_values = affc.get('landcover_class_values').getInfo()
print(class_values)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [35]:
class_palette = affc.get('landcover_class_palette').getInfo()
print(class_palette)

['3333ff', '996666', 'cc6699', 'ffff00', '993399', 'ffcc33', '92a55b', 'ffff99', 'cc9933', '006600', '00cc00', 'cc9900']


In [36]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [37]:
Map.addLayer(landcover, {}, 'Land cover')
Map

Map(center=[43, -80.55], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

### Visualize the result

In [38]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [39]:
# Map.add_legend(builtin_legend='NLCD')
# Map

### Export the result

Export the result directly to your computer:

In [40]:
# import os

# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# out_file = os.path.join(out_dir, 'landcover.tif')

In [41]:
# geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [42]:
# geemap.ee_export_image_to_drive(
#     landcover, description='landcover', folder='export', scale=900
# )